In [ ]:
##Creating RAG

retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs = {"k":1})

##RAG


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough



messages = """

Answer this question using the provided context only.

{question}

Context:

{context}

"""

prompt = ChatPromptTemplate.from_messages([("human",messages)])


rag_chain = { "context":retriever,"question":RunnablePassthrough()}|prompt|llm  

response = rag_chain.invoke("Tell me about Dogs")

print(response.content)

In [ ]:
# Pure RAG

## converting to retriever
retriever = vector_db.as_retriever()
retriever


## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


## Making Chain. combining Retrieve, Prompt, LLM

question_answer_chain = create_stuff_documents_chain(llm,prompt)

rag_chain = create_retrieval_chain(retriever,question_answer_chain)


##Asking question (Exam)

response = rag_chain.invoke({"input":"What is Self-Reflection"})
response

##############################(contin)
## RAG with Chat History (IMPORTANCE)

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


# with "system_prompt" create prompt

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## cREATING CHAIN
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

conversational_rag_chain.invoke(
    {"input":"What is Task Decomposition"},
    config={
        "configurable" : {"session_id":"abc123"}
    },
    )["answer"]

